In [8]:
import cv2 
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
import numpy as np

In [9]:
dataset_path = "../data/fashion_small_100/"

### Feature Extraction
Extraer los features de las imagenes de la carpeta utilizando opencv con SIFT.

In [25]:
sift = cv2.SIFT_create()
descriptor_list = []  # Aquí guardaremos todos los descriptores de todas las imágenes

for filename in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, filename)
    
    if os.path.isfile(file_path):
        img = cv2.imread(file_path)
        if img is None:
            print(f"No se pudo leer la imagen: {file_path}")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = sift.detectAndCompute(gray, None)

        if descriptors is not None:
            descriptor_list.extend(descriptors)  # 128 descriptores

descriptor_list = np.array(descriptor_list, dtype=np.float32) # Convertir a numpy con float32 para que no salgan doubles y de error


In [11]:
# Guardar features extraídos en un archivo

### Construcción del diccionario visual
- Se recopilan los descriptores locales extraídos de múltiples objetos (imágenes o audios)
para conformar un único conjunto de datos.
- Se aplica el algoritmo K-Means para agrupar los descriptores en clusters, donde cada
cluster representa un “visual Word” (en el caso de imágenes) o un “acoustic word” (en el
caso de audio).
- El centroide de cada cluster se considera un codeword, y el conjunto de estos codewords
conforma el diccionario visual o acústico.

In [27]:
def build_histogram(descriptors, kmeans):
    cluster_assignments = kmeans.predict(descriptors) # Predecir el cluster de cada descriptor
    
    histogram = np.zeros(kmeans.n_clusters, dtype=int) #histograma vacío (tamaño igual al numero de clusters)
    
    for idx in cluster_assignments:
        histogram[idx] += 1
    
    # Normalizar (para ayudar en la clasificación)
    histogram = histogram.astype(float)
    histogram /= np.sum(histogram)

    return histogram

In [29]:
kmeans = KMeans(n_clusters=10, random_state=0, n_init="auto").fit(descriptor_list)

all_histograms = []
sift = cv2.SIFT_create()  
for filename in os.listdir(dataset_path):  # Iterar sobre todas las imágenes de la carpeta
    file_path = os.path.join(dataset_path, filename)
    
    if os.path.isfile(file_path):
        img = cv2.imread(file_path) 
        if img is None:
            print(f"No se pudo leer la imagen: {file_path}")
            continue

        image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
        keypoints, descriptors = sift.detectAndCompute(image_gray, None)

        if (descriptors is not None):
            histogram = build_histogram(descriptors, kmeans)
            all_histograms.append(histogram)

Predecir

In [30]:
from sklearn.neighbors import NearestNeighbors

def predict_image(image_path, kmeans, all_histograms):
        image = cv2.imread(image_path)
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  

        keypoint, descriptor = sift.detectAndCompute(image_gray, None)

        histogram = build_histogram(descriptor, kmeans)
        neighbor = NearestNeighbors(n_neighbors = 20)
        neighbor.fit(all_histograms)
        dist, result = neighbor.kneighbors([histogram])

        return dist, result

In [32]:
image_path = "C:/Users/davie/Downloads/fashion_small/images/58482.jpg"
dist, result = predict_image(image_path, kmeans, all_histograms)
print(dist)
print(result)

[[0.3350297  0.35136418 0.40824829 0.41573971 0.42056004 0.42163702
  0.42599822 0.44221664 0.44411559 0.44487826 0.44659376 0.44946657
  0.45133547 0.46091811 0.47626091 0.47958315 0.48092881 0.48203262
  0.48279024 0.49236596]]
[[41 66 34 82 23 37 67  8 70 38 13 68 55 62 25 74  5 51 17 79]]


otra implementación